In [1]:
import numpy as np
from scipy.io import loadmat
df= loadmat('hw6data.mat')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
df.keys()

dict_keys(['__header__', '__version__', '__globals__', 'L', 'seq1_h', 'seq1_x', 'seq2_h', 'seq2_x'])

### Qb

In [3]:
h1=df["seq1_h"].squeeze()
x1=df["seq1_x"]

In [4]:
def calA(h,i,j):
    num=0
    for t in range(h.shape[0]-1):
        if (h[t]==i) and (h[t+1])==j:
            num+=1
    denom= np.where(h[:-1]==i,1,0).sum()
    return num/denom

In [5]:
A11 = calA(h1,1,1)
A12 = calA(h1,1,2)
A21 = calA(h1,2,1)
A22 = calA(h1,2,2)

In [6]:
print(A11,A12,A21,A22)

0.895306028532 0.104693971468 0.101497876146 0.898502123854


In [7]:
mu1 = np.average(x1[np.where(h1==1)],axis=0)
mu2 = np.average( x1[np.where(h1==2)],axis=0)

### Qc

In [8]:
h2=df["seq2_h"].squeeze()
x2=df["seq2_x"]

In [9]:
mu1

array([ 3.57880606,  0.41957754,  0.65787761,  0.01398859, -0.3751302 ,
       -1.03407044,  0.1038199 , -0.2005791 ])

In [10]:
def xmessage(mu,x):
    first = 1/np.power((2*np.pi),4)
    second = np.exp(-(1/2)*np.linalg.norm(x-mu))
    return first * second
    
def x_cond(x):
    return np.array([xmessage(mu1,x),xmessage(mu2,x)])

In [11]:
h_cond = np.array([[A11,A12],[A21,A22]])
x1_cond = x_cond(x2[0])
pi = np.array([A21,A22])

In [12]:
ph1_x1_num = np.multiply(pi,x1_cond)
ph1_x1 = ph1_x1_num/ph1_x1_num.sum()
def choose_h(p):
    if p[0]>=p[1]:
        return 1
    else:
        return 2
pred_h1 =  choose_h(ph1_x1)
pred = [pred_h1]

In [13]:
def numerator(prev,x):
    first = prev.dot(h_cond)
    return np.multiply(first, x_cond(x))

def denom(num):
    return num.sum()

In [14]:
prev = ph1_x1
for i in range(1,x2.shape[0]):
    num = numerator(prev,x2[i])
    denomen = denom(num)
    hi=choose_h(num/denomen)
    pred.append(hi)
    prev = num/denomen

In [15]:
np.where(pred!=h2,1,0).sum()

284